## MyHomeBudget

Aplikacja MyFinance wykorzystuje `TabLayout Navigation` z `ViewPager2`. Dane dostarczymy przez obiekt `dataProvider` zawierający hardcodowane przykładowe pozycje (*dummy data*). Aplikacja zawiera trzy ekrany pokazujące stan oszczędności oraz rachunki - ekran główny zawiera podsumowanie. Dodamy customową animację do `TabLayout.Tab` oraz `DonutChart` podsumowujący stan naszych finansów oraz kilka elementów urozmaicających wygląd naszej aplikacji. Sam layout aplikacji bazuje na przykładzie z **Android Open Project** [link](https://github.com/android/compose-samples/tree/main/Rally) - będzie to wersja aplikacji wcześniej zaimplementowanej w module 1a

<table><tr><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExZjYyMzE0YjY1ZmMxNzk1NjllNTY3NjgxODcxY2M5ZDY1ZWQ2YWQxNCZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/KfMbjbJosqcsJhKcti/giphy.gif" width="200" /></td><td><img src="https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExNTFlNWY1NzY5NjhjYWZiMmMxY2ZmMzQzZWE3YjM3MzZiOTRhMzAzYiZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/SOv7hdHqBXI3B9uwJP/giphy.gif" width="200" /></td><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExMmRiMWVlZmIyYmRiMjEyYTdjMDU1MWI1OGM5MTI5ZjU4NzNjMTUyOSZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/TVVqv1DfADRPbKWN4a/giphy.gif" width="200" /></td></tr></table>

Rozpocznijmy od dodania zależności do projektu - wykorzystamy `Navigation` oraz będziemy potrzebować dodatkowe ikony w projekcie

In [ ]:
implementation 'androidx.navigation:navigation-compose:2.5.3'
implementation "androidx.compose.material:material-icons-extended:1.4.3"

Nasza aplikacja będzie posiadała zdefiniowany layout tylko w orientacji `portrait`, więc zablookujemy możliwość zmiany w plik `AndroidManifest.xml` - do aktywności dodamy

In [ ]:
android:screenOrientation="portrait"

## Model

Dodajmy model danych

In [ ]:
data class Account(
    val name: String,
    val number: String,
    val amount: Double,
    val color: Int
)

In [ ]:
data class Bill (
    val name: String,
    val endDate: LocalDate,
    val amount: Double,
    val color: Int
)

Dane, podobnie jak w poprzednich przykładach, będziemy pobierać z obiektu `DataProvider`

In [ ]:
object DataProvider {

    val accounts = listOf(
        Account("Home savings", "1111111111", 23456.34, Color.BLUE),
        Account("Car savings", "2222222222", 126578.99, Color.LTGRAY),
        Account("Vacation", "3457733323", 9875.12, Color.MAGENTA),
        Account("Emergency", "9488344443", 10000.77, Color.RED),
        Account("Healthcare", "3243554434", 12345.00, Color.YELLOW),
        Account("Shopping", "2947560007", 3456.56, Color.BLACK)
    )

    val bills = listOf(
        Bill("Bank Credit", LocalDate.of(2022, 9,22), 2300.0, Color.BLACK),
        Bill("Tuition", LocalDate.of(2023, 2,10), 1200.0, Color.BLUE),
        Bill("Rent", LocalDate.of(2022, 8,3), 1023.87, Color.YELLOW),
        Bill("Loan", LocalDate.of(2022, 12,22), 334.0, Color.GRAY),
        Bill("Car Repair", LocalDate.of(2023, 1,9), 982.38, Color.WHITE),
        Bill("Dress Loan", LocalDate.of(2023, 5,18), 243.0, Color.MAGENTA)
    )
}

## Nawigacja

Nasz aplikacja składa się z trzech ekranów, dodajmy podstawowy layout

In [ ]:
@Composable
fun AccountsScreen(){
    Box(
        Modifier
            .fillMaxSize(),
        contentAlignment = Alignment.Center
    ) {
        Text(
            text = "First Screen",
            fontSize = 40.sp
        )
    }
}

@Preview(showBackground = true)
@Composable
fun FirstPreview() {
    MyHomeBudgetComposeTheme {
        AccountsScreen()
    }
}

In [ ]:
@Composable
fun BillsScreen(){
    Box(
        Modifier
            .fillMaxSize(),
        contentAlignment = Alignment.Center
    ) {
        Text(
            text = "Second Screen",
            fontSize = 40.sp
        )
    }
}

@Preview(showBackground = true)
@Composable
fun SecondPreview() {
    MyHomeBudgetComposeTheme {
        BillsScreen()
    }
}

In [ ]:
@Composable
fun OverviewScreen(){
    Box(
        Modifier
            .fillMaxSize(),
        contentAlignment = Alignment.Center
    ) {
        Text(
            text = "Home Screen",
            fontSize = 40.sp
        )
    }
}

@Preview(showBackground = true)
@Composable
fun HomePreview() {
    MyHomeBudgetComposeTheme {
        OverviewScreen()
    }
}

Dodajmy klasę reprezentującą właściwości ekranów

In [ ]:
sealed class Screens(
    val route: String,
    val icon: ImageVector
) {
    object Overview : Screens("overview", Icons.Filled.AccountBalance)
    object Accounts : Screens("accounts", Icons.Filled.AttachMoney)
    object Bills : Screens("bills", Icons.Filled.MoneyOff)
}

Następnie dodajmy komponenty składowe naszych zakładek, które umieścimy w `topBar` komponentu `Scaffold`. Rozpocznijmy od dodania funkcji pozwalającej na renderowanie pojedynczej zakładki w interfejsie użytkownika. Zakładka może być wybrana lub nieaktywna, a tekst i ikona będą animowane - animacja będzie wyświetlana w zależności od jej stanu.

In [ ]:
@Composable
private fun SingleTab(
    text: String,
    icon: ImageVector,
    onSelected: () -> Unit,
    selected: Boolean
) {}


Funkcja przyjmuje następujące parametry:
- `text` - tekst wyświetlany na zakładce,
- `icon` - ikona wyświetlana na zakładce,
- `onSelected` - funkcja wywoływana po wybraniu zakładki,
- `selected` - flaga wskazująca, czy zakładka jest wybrana.

In [ ]:
@Composable
private fun SingleTab(
    text: String,
    icon: ImageVector,
    onSelected: () -> Unit,
    selected: Boolean
) {

    val color = MaterialTheme.colorScheme.secondary
    val durationMillis = if (selected) TabFadeInAnimationDuration else TabFadeOutAnimationDuration
    val animSpec = remember {
        tween<Color>(
            durationMillis = durationMillis,
            easing = LinearEasing,
            delayMillis = TabFadeInAnimationDelay
        )
    }
    val tabTintColor by animateColorAsState(
        targetValue = if (selected) color else color.copy(alpha = InactiveTabOpacity),
        animationSpec = animSpec
    )
}

private val TabHeight = 56.dp
private const val InactiveTabOpacity = 0.60f

private const val TabFadeInAnimationDuration = 150
private const val TabFadeInAnimationDelay = 100
private const val TabFadeOutAnimationDuration = 100

Definijuemy kilka stałych:
- `color` - kolor używany do tła i tekstu zakładki, pobierany z `MaterialTheme`,
- `durationMillis` - czas trwania animacji dla zakładki, zależny od stanu wybrania zakładki,
- `animSpec` - specyfikacja animacji używanej do animowania koloru zakładki,
- `tabTintColor` - kolor tła i tekstu zakładki, który jest animowany na podstawie stanu wybrania zakładki.

W stałej `animSpec` przechowujemy specyfikację animacji, która zostanie użyta do animowania koloru zakładki. Dzięki zastosowaniu `remember`, specyfikacja animacji jest zapamiętana i nie jest obliczana przy każdym ponownym renderowaniu komponentu. Wewnątrz bloku `remember` tworzymy instancję `tween<Color>`, która jest specyfikacją animacji, działającą na wartości koloru. `tween` to funkcja, która tworzy animację opartą na interpolacji. Przyjmuje ona kilka parametrów:
- `durationMillis` - czas trwania animacji w milisekundach,
- `easing` - funkcja określająca sposób interpolacji wartości animacji (np. `LinearEasing` dla liniowej interpolacji),
- `delayMillis` - opóźnienie animacji przed jej rozpoczęciem w milisekundach.

Gdy zostanie uruchomiona animacja zmiany koloru zakładki, wykorzystamy `animSpec` do kontrolowania jej przebiegu.

Następnie dodajemy `tabColorTint`, który umożliwia animowanie koloru zakładki na podstawie wartości `selected`. Dzięki użyciu `animateColorAsState` i odpowiedniej specyfikacji animacji (`animSpec`), kolor zakładki płynnie zmienia się wraz ze zmianą stanu. 

`animateColorAsState` to funkcja, tworząca stan (`State<Color>`), który może być animowany. Przyjmuje ona dwa parametry:
- `targetValue` - wartość docelowa, do której kolor ma być animowany. Jeśli `selected` ma wartość `true`, kolor zostanie ustawiony na `color` (`MaterialTheme.colorScheme.secondary`), w przeciwnym razie zostanie użyta kopia `color` z zastosowanym zmniejszeniem przez `InactiveTabOpacity` (wartość przezroczystości dla nieaktywnej zakładki)
- `animationSpec` - specyfikacja animacji, która określa czas trwania, opóźnienie i sposób interpolacji animacji.

`tabTintColor` jest wartością zwracaną typu `State<Color>`, który reprezentuje bieżącą wartość koloru zakładki. Kiedy wartość `targetValue` zmienia się na skutek animacji, `tabTintColor` również się zmienia, co powoduje ponowne renderowanie komponentu.

W komponencie używamy `Row`, który definiuje zawartość i wygląd pojedynczej zakładki.

In [ ]:
@Composable
private fun SingleTab(
    text: String,
    icon: ImageVector,
    onSelected: () -> Unit,
    selected: Boolean
) {

    val color = MaterialTheme.colorScheme.secondary
    val durationMillis = if (selected) TabFadeInAnimationDuration else TabFadeOutAnimationDuration
    val animSpec = remember {
        tween<Color>(
            durationMillis = durationMillis,
            easing = LinearEasing,
            delayMillis = TabFadeInAnimationDelay
        )
    }
    val tabTintColor by animateColorAsState(
        targetValue = if (selected) color else color.copy(alpha = InactiveTabOpacity),
        animationSpec = animSpec
    )
    Row(
        modifier = Modifier
            .padding(16.dp)
            .animateContentSize()
            .height(TabHeight)
            .selectable(
                selected = selected,
                onClick = onSelected,
                role = Role.Tab,
                interactionSource = remember { MutableInteractionSource() },
                indication = rememberRipple(
                    bounded = false,
                    radius = Dp.Unspecified,
                    color = Color.Unspecified
                )
            )
            .clearAndSetSemantics { contentDescription = text },
        content =  {
            Icon(imageVector = icon, contentDescription = text, tint = tabTintColor)
            if (selected) {
                Spacer(Modifier.width(12.dp))
                Text(text.uppercase(Locale.getDefault()), color = tabTintColor)
            }
        }
    )
}

private val TabHeight = 56.dp
private const val InactiveTabOpacity = 0.60f

private const val TabFadeInAnimationDuration = 150
private const val TabFadeInAnimationDelay = 100
private const val TabFadeOutAnimationDuration = 100

W modyfikatorze `Row` `animateContentSize()` animuje zmianę rozmiaru wiersza na podstawie zawartości, `selectable` to modifikator, który umożliwia zaznaczanie wiersza. Przyjmuje kilka parametrów:
- `selected` - wartość logiczna, czy dany wiersz jest zaznaczony czy nie,

- `onClick` - funkcja wywoływana po kliknięciu wiersza, w tym przypadku wywołuje funkcję `onSelected`, która jest przekazywana jako parametr,

- `role` - rola wiersza, w tym przypadku jest to rola `Role.Tab`, co wskazuje na to, że wiersz reprezentuje zakładkę - można nadać zakładce wiele różnych ról: 
    - `Role.Tab` - rola oznaczająca, że wiersz reprezentuje zakładkę.
    - `Role.Button` - rola oznaczająca, że wiersz reprezentuje przycisk.
    - `Role.Checkbox` - rola oznaczająca, że wiersz reprezentuje pole wyboru (*checkbox*).
    - `Role.RadioButton` - rola oznaczająca, że wiersz reprezentuje przycisk opcji (*radio button*).
<br/><br/>
 
- `interactionSource` - źródło interakcji, w tym przypadku jest to nowa instancja `MutableInteractionSource` - jest to źródło interakcji, które może być używane do rejestrowania i odczytywania zdarzeń interakcji. Przechowuje informacje o bieżących stanach interakcji, takich jak naciśnięcie, dotknięcie, przeciągnięcie, wybór itp. Dzięki `MutableInteractionSource` możemy monitorować i reagować na różne zdarzenia interakcji w naszych komponentach Compose. Możemy przekazać MutableInteractionSource do odpowiednich komponentów, takich jak `Clickable`, `Selectable`, `Draggable`, aby obsłużyć i reagować na interakcje użytkownika. Na przykład, możemy śledzić kliknięcia na wiersz, monitorować stan naciśnięcia lub odznaczenia w polu wyboru, itp.

- `indication` - efekt wizualny podczas interakcji z wierszem, w tym przypadku używamy efektu *ripple* (falowania) za pomocą funkcji `rememberRipple`. Funkcja ta zwraca obiekt `RippleIndication` zawierający informacje o zdefiniowanym efekcie falowania. Następnie przekazujemy ten obiekt do modifikatora `selectable`.
    - `bounded` Określa, czy efekt falowania ma być ograniczony do granic wiersza (`true`), czy ma wypełnić całą przestrzeń wokół wiersza (`false`).
    - `radius` Określa promień efektu falowania. Może przyjąć wartość `Dp.Unspecified` dla dynamicznego określenia promienia na podstawie rozmiaru wiersza.
    - `color` Określa kolor efektu falowania. Może być używany kolor zdefiniowany wcześniej lub `Color.Unspecified` dla automatycznego dopasowania koloru do motywu.
    <br/><br/>
- `clearAndSetSemantics` to modifikator, który ustawia opis treści (semantykę) dostępności dla wiersza na wartość `text`, która jest przekazywana jako parametr. Jest używane głównie w celu dostarczenia informacji o dostępności i ułatwienia korzystania z interfejsu użytkownika dla osób korzystających z technologii asystujących.
    - Opisy obrazków: Możemy użyć do ustawienia opisu (`contentDescription`) dla obrazków w celu opisania ich zawartości osobom niewidomym lub słabowidzącym.
    - Tekst alternatywny dla elementów interaktywnych: Możemy ustawić tekst alternatywny (`contentDescription`) dla elementów interaktywnych, takich jak przyciski lub linki, aby przekazać informację o ich funkcji użytkownikom, którzy nie widzą lub mają trudności z odczytaniem etykiet.
    - Dostępność treści: Możemy użyć do dostarczenia informacji o dostępności dla różnych elementów treści, takich jak nagłówki, akapity, listy, tabelki, itp.
    </br></br>
    
- `content` - zawartość wiersza
    - `Icon` reprezentuje ikonę zakładki. Przyjmuje trzy parametry:
        - `imageVector` Obraz wektorowy (np. ikona), który ma być wyświetlany jako ikona zakładki.
        - `contentDescription` Opis treści ikony, który jest przekazywany w celach dostępności.
        - `tint` Kolor, który ma być zastosowany do ikony zakładki.
        </br></br>
    - Warunek `if (selected)` Jest to warunek sprawdzający, czy dany wiersz jest zaznaczony. Jeśli tak, to wyświetlany jest dodatkowy element wiersza.
    - `Spacer(Modifier.width(12.dp))` Komponent używany jest do utworzenia pustej przestrzeni pomiędzy ikoną a tekstem.
    - `Text` Ten komponent reprezentuje tekst zakładki. Przyjmuje dwa parametry:
        - `text` Treść, która ma być wyświetlana jako tekst zakładki.
        - `color` Kolor tekstu zakładki.
    </br></br>
    Oznacza to, że jeśli wiersz jest zaznaczony (selected), zostanie wyświetlony tekst zakładki obok ikony, z odpowiednimi odstępami i kolorem tekstu zdefiniowanym przez tabTintColor. Jeśli wiersz nie jest zaznaczony, tylko ikona zakładki będzie widoczna.

Zdefiniujmy komponent reprezentujący wiersz z zakładkami.

In [ ]:
@Composable
fun TabRow(
    allScreens: List<Screens>,
    onTabSelected: (Screens) -> Unit,
    currentScreen: Screens
) {
    Surface(
        Modifier
            .height(TabHeight)
            .fillMaxWidth()
    ) {
        Row(Modifier.selectableGroup()) {
            allScreens.forEach { screen ->
                SingleTab(
                    text = screen.route,
                    icon = screen.icon,
                    onSelected = { onTabSelected(screen) },
                    selected = currentScreen == screen
                )
            }
        }
    }
}

Komponent ten przyjmuje trzy parametry:
- `allScreens` Lista wszystkich dostępnych zakładek (obiektów typu `Screens`).
- `onTabSelected` Funkcja, która jest wywoływana po wybraniu zakładki. Przyjmuje jako argument wybraną zakładkę.
- `currentScreen` Obecnie wybrana zakładka.


Komponent `TabRow` tworzy wiersz zakładek używając komponentów `Surface` i `Row`. Wewnątrz wiersza iteruje po wszystkich zakładkach z listy `allScreens` i dla każdej zakładki renderuje komponent `SingleTab`. `Surface` tworzy powierzchnię, która otacza wiersz zakładek. `Row` tworzy poziomy układ, w którym są renderowane poszczególne zakładki. Używa on również modyfikatora `selectableGroup()`, który umożliwia zaznaczanie jednego elementu wiersza. `onSelected` to funkcja, która jest wywoływana po wybraniu zakładki. `selected` informuje, czy dana zakładka jest obecnie wybrana (porównuje się ją z `currentScreen`).

W efekcie, komponent `TabRow` renderuje wiersz zakładek, gdzie każda zakładka jest reprezentowana przez komponent `SingleTab`, a wybrana zakładka ma odpowiednie oznaczenie graficzne. Po kliknięciu na zakładkę wywoływana jest funkcja `onTabSelected` z wybraną zakładką jako argumentem.

Następnie tworzymy nawigację, rozpoczniemy od utworzeniu grafu nawigacji.

In [ ]:
@Composable
fun NavGraph(navController: NavHostController, modifier: Modifier){
    NavHost(
        navController = navController,
        startDestination = Screens.Overview.route,
        modifier = modifier
    ) {
        composable(route = Screens.Overview.route){ OverviewScreen() }
        composable(route = Screens.Accounts.route){ AccountsScreen() }
        composable(route = Screens.Bills.route){ BillsScreen() }
    }
}

Aby uniknąć problemu gromadzenia dużej liczby ekranów na stosie nawigacji, w miarę wybierania przez użytkowników elementów, dodajmy funkcję rozszerzającą zapewniającą zachowanie `singleTop`

In [ ]:
fun NavHostController.navigateSingleTopTo(route: String) =
    this.navigate(route) {
        // Pop up to the start destination of the graph to
        // avoid building up a large stack of destinations
        // on the back stack as users select items
        popUpTo(
            this@navigateSingleTopTo.graph.findStartDestination().id
        ) {
            saveState = true
        }
        // Avoid multiple copies of the same destination when
        // reselecting the same item
        launchSingleTop = true
        // Restore state when reselecting a previously selected item
        restoreState = true
}

Przed wywołaniem nawigacji, zdefiniowane są pewne ustawienia nawigacji typu:
- `popUpTo` Ustala, że należy cofnąć się do startowego celu grafu, aby uniknąć gromadzenia dużej ich liczby na stosie nawigacji. W przypadku nawigacji do nowego celu, stare zostaną usunięte z powrotem do punktu startowego. Ta konfiguracja jest osiągana poprzez wywołanie metody `popUpTo` i przekazanie identyfikatora startowego celu grafu (odnaleziony poprzez `this@navigateSingleTopTo.graph.findStartDestination().id`).
- `launchSingleTop` Ustala, że nawigacja do tego samego celiu, która jest już wybrana, nie spowoduje tworzenia nowej instancji tego celu. Zamiast tego, zostanie przywrócona już istniejąca. Ta konfiguracja jest osiągana przez ustawienie wartości `launchSingleTop` na `true`.
- `restoreState` Ustala, że stan poprzednio wybranego celu zostanie przywrócony, jeśli ten sam cel zostanie ponownie wybrany. W efekcie, użytkownikowi zostanie przedstawiony ostatni stan tego celu, a nie będzie konieczności budowania nowego stanu. Ta konfiguracja jest osiągana przez ustawienie wartości `restoreState` na `true`.

Ostatnią funkcję w tej części jest `Navigation`, odpowiedzialna za nawigację.

In [ ]:
@OptIn(ExperimentalMaterial3Api::class)
@Composable
fun Navigation(){
    val navController = rememberNavController()
    val screens = listOf(
        Screens.Overview, Screens.Accounts, Screens.Bills
    )

    val navBackStackEntry by navController.currentBackStackEntryAsState()
    val currentDestination = navBackStackEntry?.destination
    val currentScreen =
        screens.find { it.route == currentDestination?.route } ?: Screens.Overview

    Scaffold(
        topBar = {
            TabRow(
                allScreens = screens,
                onTabSelected = { newScreen ->
                    navController.navigateSingleTopTo(newScreen.route)
                },
                currentScreen = currentScreen
            )
        }
    ) { paddingValues ->
        NavGraph(
            navController = navController,
            modifier = Modifier.padding(paddingValues)
        )
    }
}

Bieżący ekran (`currentScreen`) odnajdujemy na podstawie wartości `currentDestination` i listy `screens`. Jeżeli nie zostanie znaleziony pasujący ekran, przyjmuje się ekran startowy (`Screens.Overview`). Następnie renderowany jest `Scaffold` z górnym paskiem (`topBar`) zawierającym `TabRow`. Otrzymuje on listę wszystkich dostępnych ekranów (`screens`), funkcję obsługującą zmianę wybranego ekranu (`onTabSelected`) oraz informację o bieżącym ekranie (`currentScreen`). Przy zmianie ekranu wykorzystywana jest funkcja `navController.navigateSingleTopTo()`, która odpowiada za nawigację do nowego ekranu w trybie *single top* - czyli cofnięcie się do punktu startowego nawigacji i uniknięcie duplikacji na stosie nawigacji. Na koniec renderowany jest `NavGraph`, który obsługuje resztę nawigacji w aplikacji. Przekazany jest do niego `navController` oraz `paddingValues` w celu odpowiedniego wcięcia treści w obszarze głównym (`content`) `Scaffold`.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExODA2MDVlNDU5M2Y5MzdlNWQyNjZlODE0YTRjMWRlNDBiYjgzNmVjNSZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/1wWu0Rv6XAYaWhHmQr/giphy.gif" width="200" />

## Ekran rachunków

### `DonutChart`

Wpierw wycentrujmy zakładki na górnej belce

In [ ]:
@Composable
fun TabRow(
    allScreens: List<Screens>,
    onTabSelected: (Screens) -> Unit,
    currentScreen: Screens
) {
    Surface(
        Modifier
            .height(TabHeight)
            .fillMaxWidth()
    ) {
        Row(Modifier.selectableGroup(), horizontalArrangement = Arrangement.Center) {
            allScreens.forEach { screen ->
                SingleTab(
                    text = screen.route,
                    icon = screen.icon,
                    onSelected = { onTabSelected(screen) },
                    selected = currentScreen == screen
                )
            }
        }
    }
}

W komponencie `Row` dodajemy `horizontalArrangement = Arrangement.Center`.

<img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExZTBmNjI3ODUxYWQ3MDg2YWEzZGY5NjA5MTM2ZmMxMTUzYjVkOTE0NSZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/biWp9vC7NmqrepkLXg/giphy.gif" width="200" />

Następnie zmodyfikujmy nieco nasz model danych.

In [ ]:
import androidx.compose.ui.graphics.Color
import java.time.LocalDate

data class Bill (
    val name: String,
    val endDate: LocalDate,
    val amount: Double,
    val color: Color
)

In [ ]:
import androidx.compose.ui.graphics.Color

data class Account(
    val name: String,
    val number: String,
    val amount: Double,
    val color: Color
)

Pole `color` jest typu `Color` z biblioteki `compose.ui.graphics.Color`, zamiast typu `Int` z biblioteki `graphics` - również musimy zmienić nazwy kolorów w `DataProvider`

In [ ]:
object DataProvider {

    val accounts = listOf(
        Account("Home savings", "1111111111", 23456.34, Color.Blue),
        Account("Car savings", "2222222222", 126578.99, Color.LightGray),
        Account("Vacation", "3457733323", 9875.12, Color.Magenta),
        Account("Emergency", "9488344443", 10000.77, Color.Red),
        Account("Healthcare", "3243554434", 12345.00, Color.Yellow),
        Account("Shopping", "2947560007", 3456.56, Color.Black)
    )

    val bills = listOf(
        Bill("Bank Credit", LocalDate.of(2022, 9,22), 2300.0, Color.Black),
        Bill("Tuition", LocalDate.of(2023, 2,10), 1200.0, Color.Blue),
        Bill("Rent", LocalDate.of(2022, 8,3), 1023.87, Color.Yellow),
        Bill("Loan", LocalDate.of(2022, 12,22), 334.0, Color.Gray),
        Bill("Car Repair", LocalDate.of(2023, 1,9), 982.38, Color.White),
        Bill("Dress Loan", LocalDate.of(2023, 5,18), 243.0, Color.Magenta)
    )

    val totalAccountsAmount = (accounts.indices).sumOf { accounts[it].amount }
    val totalBillsAmount = (bills.indices).sumOf { bills[it].amount }
}

Możemy przystąpić do tworzenia layoutu dla ekranu prezentującego rachunki. Rozpoczniemy od `DonutChart`. Wykorzystamy proste animacje. Efekt końcowy będzie prezentował się następująco:

<img src="https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExMGUxOTU0ZDE2NzgzZjljZjgwN2QzMzdlYTI1OThmZTYzNjgyY2M0MSZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/SsomlDGvA15rM2M2nz/giphy.gif" width="200" />

Rozpocznijmy od utworzenia komponentu `AnimatedCircle`

In [ ]:
@Composable
fun AnimatedCircle(
    proportions: List<Float>,
    colors: List<Color>,
    modifier: Modifier = Modifier
) {}

Funkcja przyjmuje listę liczb `Float` - są to proporcje naszego `DonutChart` (gdy przekażemy listę `listOf(0.5f, 0.5f)` okrąg zostanie podzielony na dwa obszary o róznych proporcjach), proporcje muszą sumować się do `1.0`. Drugim parametrem jest lista kolorów, które zostaną wykorzystane do narysowania okręgu. Ostatnim parametrem jest `Modifier` dzięki któremu możemy zmodyfikować wygląd i zachowanie.

Następnie tworzymy `currentState` przy użyciu `remember`, który przechowuje stan przejścia animacji.

In [ ]:
val currentState = remember {
    MutableTransitionState(AnimatedCircleProgress.START)
        .apply { targetState = AnimatedCircleProgress.END }
}

Tworzymy nowy obiekt `MutableTransitionState` i przekazujemy do niego początkowy stan animacji (`AnimatedCircleProgress.START`). Następnie wywołujemy funkcję `apply` na tym obiekcie i ustawiamy docelowy stan animacji na `AnimatedCircleProgress.END`.

`MutableTransitionState` umożliwia kontrolowanie przejścia między stanami za pomocą metod `targetState` (ustawia docelowy stan animacji) i `animateTo` (uruchamia animację między bieżącym a docelowym stanem). W tym przypadku, przy użyciu `remember`, zapamiętujemy ten obiekt, aby zachować jego stan pomiędzy kolejnymi renderowaniami.

Potrzebujemy również obiekt `Stroke` do reprezentowania linii rysowanego okręgu.

In [ ]:
val stroke = with(LocalDensity.current) { Stroke(8.dp.toPx()) }

Tworzymy obiekt `Stroke` z grubością linii `8 dp`. Aby przeliczyć wartość na piksele, korzystamy z `LocalDensity.current`, który dostarcza informacji o aktualnej gęstości ekranu.

W kolejnym kroku tworzymy obiekt `Transition`, jest to mechanizm, który umożliwia animowanie między różnymi stanami. Pozwala na płynne przejście pomiędzy dwoma stanami i kontrolowanie animacji na podstawie tych stanów. Dzięki tej klasie, można tworzyć interaktywne animacje, które reagują na zmiany stanów aplikacji, takie jak kliknięcia, zmiana danych itp. za pomocą jej metod. Składniki klasy:

- `State` - Reprezentuje bieżący stan animacji. Może to być pojedyncza wartość, lista wartości lub bardziej zaawansowane modele stanu.
- `Animation` - Definiuje sposób, w jaki animacja ma być wykonywana. Określa czas trwania, krzywą animacji, opóźnienie itp.
- `Target Values` - Określa docelowe wartości, do których animacja ma zmierzać. Może to być wartość pojedyncza, lista wartości lub funkcja transformująca wartość.

In [ ]:
val transition = updateTransition(currentState, label = "")

Metoda `updateTransition` tworzy `Transition` i umieszcza go w bieżącym stanie (`currentState`) dostarczonego `transitionState`. Za każdym razem, gdy `targetState` stanu `transitionState` ulega zmianie, `Transition` zostanie animowany do nowego stanu docelowego.

W bloku `Canvas`, rysowany jest będzie nasz wykres na podstawie obliczonych wartości `angleOffset` i `shift`. 
- `angleOffset` określa kąt, o który przesunięte są poszczególne sekcje wykresu w trakcie animacji. 
- `shift` określa przesunięcie kątowe całego wykresu.

W kolejnym kroku zdefiniujmy wartość `angleOffset`

In [ ]:
val angleOffset by transition.animateFloat(
    transitionSpec = {
        tween(
            delayMillis = 500,
            durationMillis = 900,
            easing = LinearOutSlowInEasing
        )
    }, label = ""
) { progress -> if (progress == AnimatedCircleProgress.START) 0f else 360f }

Definiujemy wartość `angleOffset` przy użyciu funkcji `animateFloat` dostępnej na obiekcie `transition`. Jest zdefiniowana przy użyciu specyfikacji `transitionSpec`, która określa opóźnienie, czas trwania i krzywą przejścia animacji.

`angleOffset` jest animowana od wartości początkowej do wartości końcowej w zależności od postępu animacji wykresu. Jeśli progres jest równy `AnimatedCircleProgress.START`, `angleOffset` przyjmuje wartość `0f`, a w przeciwnym razie przyjmuje wartość `360f`. Cała animacja będzie trwała `900` milisekund, z opóźnieniem `500` milisekund, i zostanie zastosowana krzywa przejścia `LinearOutSlowInEasing`.

`LinearOutSlowInEasing` to krzywa przejścia animacji, która początkowo przyspiesza liniowo, a następnie stopniowo zwalnia, tworząc efekt płynnego zatrzymywania animacji. Oznacza to, że na początku animacji obiekt będzie poruszał się równomiernie, a potem stopniowo zwalnia, dając efekt płynnego zakończenia animacji.

Zdefiniujmy wartość `shift`

In [ ]:
val shift by transition.animateFloat(
    transitionSpec = {
        tween(
            delayMillis = 500,
            durationMillis = 900,
            easing = CubicBezierEasing(0f, 0.75f, 0.35f, 0.85f)
        )
    }, label = ""
) { progress -> if (progress == AnimatedCircleProgress.START) 0f else 30f }

`shift` jest wartością liczby zmiennoprzecinkowej, która jest kontrolowana przez `transition`. Wartość ta zmienia się w czasie, w zależności od wartości `progress` w `transition`. `transition.animateFloat()` tworzy animację dla zmiennej `shift`. `transitionSpec` definiuje specyfikację przejścia, czyli sposób, w jaki wartość `shift` zmienia się w czasie. W tym przypadku użyto funkcji `tween()`, która definiuje animację o opóźnieniu `500` milisekund, trwającej `900` milisekund i korzystającej z funkcji `CubicBezierEasing()`.

Funkcja `tween` służy do definiowania interpolacji animacji. Przyjmuje ona parametry: czas trwania animacji, opóźnienie, funkcję interpolacji, oraz ewentualne wartości początkowe i końcowe animacji. Głównym zadaniem funkcji jest określenie, w jaki sposób wartość animowanej właściwości zmienia się w czasie. Może to być płynne przejście pomiędzy dwoma wartościami, efekt opóźnienia, przyspieszenie, zwolnienie, lub dowolna inna nieliniowa zmiana wartości.

`CubicBezierEasing()` to funkcja, która tworzy krzywą *Bezierową* o podanych punktach kontrolnych. W tym przypadku użyto wartości (`0f, 0.75f, 0.35f, 0.85f`) jako punkty kontrolne. Ta krzywa przejścia daje efekt, w którym animacja przyspiesza na początku, a następnie zwalnia stopniowo. Jeśli progress ma wartość `AnimatedCircleProgress.START`, to shift wynosi `0f`, w przeciwnym razie wynosi `30f`. Oznacza to, że wartość shift zależy od aktualnej wartości progress i zostanie odpowiednio animowana przez transition.

Pozostaje wykorzystać utworzone elementy i zdefiniować obiekt `Canvas`. Wewnątrz bloku `Canvas` można używać różnych metod rysujących, takich jak `drawLine`, `drawRect`, `drawCircle`, `drawPath`, które pozwalają na tworzenie różnych kształtów i figur geometrycznych.

In [ ]:
Canvas(modifier) {
    val innerRadius = (size.minDimension - stroke.width) / 2
    val halfSize = size / 2.0f
    val topLeft = Offset(
        halfSize.width - innerRadius,
        halfSize.height - innerRadius
    )
    val size = Size(innerRadius * 2, innerRadius * 2)
    var startAngle = shift - 90f
    proportions.forEachIndexed { index, proportion ->
        val sweep = proportion * angleOffset
        drawArc(
            color = colors[index],
            startAngle = startAngle + DividerLengthInDegrees / 2,
            sweepAngle = sweep - DividerLengthInDegrees,
            topLeft = topLeft,
            size = size,
            useCenter = false,
            style = stroke
        )
        startAngle += sweep
    }
}

In [ ]:
private enum class AnimatedCircleProgress { START, END }
private const val DividerLengthInDegrees = 1.8f

Wartość `innerRadius` reprezentuje promień wewnętrznego okręgu, na którym będą rysowane łuki. Jest obliczany na podstawie minimalnego wymiaru `canvas` (`size.minDimension`) i szerokości linii (`stroke.width`). `halfSize` to połowa rozmiaru, a `topLeft` to punkt początkowy dla rysowania łuków. Są one obliczane na podstawie `halfSize.width`, `halfSize.height` oraz `innerRadius`. 

Obiekt `size` reprezentuje wymiary prostokąta, w którym będą rysowane łuki. Jego szerokość i wysokość wynoszą `innerRadius * 2`, co daje kwadratowy obszar o boku równym średnicy wewnętrznego okręgu. `startAngle` to kąt początkowy dla pierwszego łuku. Jest to zmienna, która zmienia się w każdej iteracji pętli, a jej wartość jest aktualizowana na podstawie `shift`. Pętla `proportions.forEachIndexed` iteruje po elementach listy `proportions`, która zawiera proporcje dla poszczególnych łuków. W każdej iteracji obliczany jest `sweep`, czyli kąt, na jaki łuk zostanie narysowany, na podstawie `proportion` i `angleOffset`.

Metoda `drawArc` służy do rysowania łuku. Po narysowaniu łuku, `startAngle` jest aktualizowany przez dodanie do niego wartości `sweep`. To umożliwia rysowanie kolejnych łuków w odpowiednich pozycjach.

Pełny kod:

In [ ]:
private const val DividerLengthInDegrees = 1.8f

/**
 * A donut chart that animates when loaded.
 */
@Composable
fun AnimatedCircle(
    proportions: List<Float>,
    colors: List<Color>,
    modifier: Modifier = Modifier
) {
    val currentState = remember {
        MutableTransitionState(AnimatedCircleProgress.START)
            .apply { targetState = AnimatedCircleProgress.END }
    }
    val stroke = with(LocalDensity.current) { Stroke(8.dp.toPx()) }
    val transition = updateTransition(currentState, label = "")
    val angleOffset by transition.animateFloat(
        transitionSpec = {
            tween(
                delayMillis = 500,
                durationMillis = 900,
                easing = LinearOutSlowInEasing
            )
        }, label = ""
    ) { progress -> if (progress == AnimatedCircleProgress.START) 0f else 360f }

    val shift by transition.animateFloat(
        transitionSpec = {
            tween(
                delayMillis = 500,
                durationMillis = 900,
                easing = CubicBezierEasing(0f, 0.75f, 0.35f, 0.85f)
            )
        }, label = ""
    ) { progress -> if (progress == AnimatedCircleProgress.START) 0f else 30f }

    Canvas(modifier) {
        val innerRadius = (size.minDimension - stroke.width) / 2
        val halfSize = size / 2.0f
        val topLeft = Offset(
            halfSize.width - innerRadius,
            halfSize.height - innerRadius
        )
        val size = Size(innerRadius * 2, innerRadius * 2)
        var startAngle = shift - 90f
        proportions.forEachIndexed { index, proportion ->
            val sweep = proportion * angleOffset
            drawArc(
                color = colors[index],
                startAngle = startAngle + DividerLengthInDegrees / 2,
                sweepAngle = sweep - DividerLengthInDegrees,
                topLeft = topLeft,
                size = size,
                useCenter = false,
                style = stroke
            )
            startAngle += sweep
        }
    }
}
private enum class AnimatedCircleProgress { START, END }


### Generic Composable

Ponieważ ekrany prezentujące rachunki oraz stan kont wyglądają podobnie - zawierają `DonutChart` i pod nim listę elementów - utworzymy komponent generyczny, który wykorzystamy na obu ekranach.

In [ ]:
@Composable
fun <T> StatementBody(
    modifier: Modifier = Modifier,
    items: List<T>,
    colors: (T) -> Color,
    amounts: (T) -> Float,
    amountsTotal: Float,
    circleLabel: String,
    rows: @Composable (T) -> Unit
) {

Parametry funkcji `StatementBody`:
- `modifier` - opcjonalny parametr, który umożliwia dostosowanie modifikatorów dla sekcji.
- `items` - lista elementów, które będą wyświetlane w sekcji.
- `colors` - funkcja, która na podstawie elementu `T` zwraca kolor, który będzie używany do stylizacji danego elementu.
- `amounts` - funkcja, która na podstawie elementu `T` zwraca wartość liczbową reprezentującą kwotę danego elementu.
- `amountsTotal` - suma wszystkich wartości liczbowych dla wszystkich elementów na liście.
- `circleLabel` - etykieta, która będzie wyświetlana w okręgu.
- `rows` - funkcja komponująca, która definiuje sposób wyświetlania pojedynczego elementu `T` na liście.

Rozpocznijmy od zdefiniowania `Column` w którym umieścimy `DonutChart` i pod nim listę elementów.

In [ ]:
Column(modifier = modifier.verticalScroll(rememberScrollState())) {

Tutaj zastosujemy modifikator `verticalScroll(rememberScrollState())`, który umożliwia przewijanie wertykalne (pionowe) dla zawartości kolumny. `rememberScrollState()` jest funkcją, która tworzy i przechowuje wewnętrzny stan przewijania dla komponentu, co pozwala na kontrolowanie przewijania wertykalnego wewnątrz `Column`. Wszystkie elementy zawarte wewnątrz `Column` będą wyświetlane pionowo, a jeśli zawartość przekracza dostępną przestrzeń, użytkownik będzie mógł przewijać ją wertykalnie.

Nasz `DonutChart` wraz z dwoma polami tekstowymi umieścimy wewnątrz `Box`. Pola tekstowe będą wyświetlać aktualną sumę wszystkich elementów listy, które będą pokazane na `DonutChart`

In [ ]:
Box(Modifier.padding(16.dp)) {
    val accountsProportion = items.extractProportions { amounts(it) }
    val circleColors = items.map { colors(it) }
    AnimatedCircle(
        accountsProportion,
        circleColors,
        Modifier
            .height(300.dp)
            .align(Alignment.Center)
            .fillMaxWidth()
    )
    Column(modifier = Modifier.align(Alignment.Center)) {
        Text(
            text = circleLabel,
            modifier = Modifier.align(Alignment.CenterHorizontally),
            fontSize = 20.sp
        )
        Text(
            text = ("- ${formatter.format(amountsTotal)} zł"),
            modifier = Modifier.align(Alignment.CenterHorizontally),
            fontSize = 36.sp
        )
    }
}

Na podstawie przekazanych elementów `items`, obliczane są proporcje kont/rachunków i tworzone są kolory dla `DonutChart`. 

In [ ]:
val accountsProportion = items.extractProportions { amounts(it) }

Funkcja `extractProportions` przyjmuje jako argument funkcję selektora, która określa, jaką wartość liczbową należy wybrać dla każdego elementu listy. Na podstawie tej wartości zostaną obliczone proporcje.

In [ ]:
fun <E> List<E>.extractProportions(selector: (E) -> Float): List<Float> {
    val total = this.sumOf { selector(it).toDouble() }
    return this.map { (selector(it) / total).toFloat() }
}

Funkcja sumuje wszystkie wartości liczbowe dla wszystkich elementów listy przy użyciu `sumOf` i przypisuje tę wartość do zmiennej `total`. Następnie, dla każdego elementu listy, funkcja dzieli wartość liczbową wybraną przez selektor przez sumę wszystkich wartości i przekształca ją na typ `Float`. Wynikowe proporcje są zwracane jako nowa lista.

Komponent `AnimatedCircle` otrzymuje te proporcje i kolory, a także modifikator `Modifier.height(300.dp).align(Alignment.Center).fillMaxWidth()`, który nadaje mu wysokość `300` dp, wyśrodkowuje go w pionie i rozciąga na maksymalną szerokość dostępną w kontenerze.

Wewnątrz `Box` znajduje się również `Column`, który jest wyśrodkowany w pionie przy pomocy modifikatora `Modifier.align(Alignment.Center)`. Wewnątrz znajdują się dwie etykiety tekstowe `Text`. Pierwsza etykieta wyświetla wartość `circleLabel` i jest wyśrodkowana w poziomie. Druga etykieta wyświetla wartość - `${formatter.format(amountsTotal)} zł` (gdzie formatter to obiekt odpowiedzialny za formatowanie liczby).

In [ ]:
val formatter: NumberFormat = DecimalFormat("#,###.##")

Następnie dodamy odstęp między `DonutChart` a listą za pomocą `Spacer`

In [ ]:
Spacer(Modifier.height(10.dp))

Listę elementów wyświetlimy w `Column`, który będzie umieszczony w `Card`

In [ ]:
Card {
    Column(modifier = Modifier.padding(12.dp)) {
        items.forEach { item ->
            rows(item)
        }
    }

Pełny kod:

In [ ]:
@Composable
fun <T> StatementBody(
    modifier: Modifier = Modifier,
    items: List<T>,
    colors: (T) -> Color,
    amounts: (T) -> Float,
    amountsTotal: Float,
    circleLabel: String,
    rows: @Composable (T) -> Unit
) {
    Column(modifier = modifier.verticalScroll(rememberScrollState())) {
        Box(Modifier.padding(16.dp)) {
            val accountsProportion = items.extractProportions { amounts(it) }
            val circleColors = items.map { colors(it) }
            AnimatedCircle(
                accountsProportion,
                circleColors,
                Modifier
                    .height(300.dp)
                    .align(Alignment.Center)
                    .fillMaxWidth()
            )
            Column(modifier = Modifier.align(Alignment.Center)) {
                Text(
                    text = circleLabel,
                    modifier = Modifier.align(Alignment.CenterHorizontally),
                    fontSize = 20.sp
                )
                Text(
                    text = ("- ${formatter.format(amountsTotal)} zł"),
                    modifier = Modifier.align(Alignment.CenterHorizontally),
                    fontSize = 36.sp
                )
            }
        }
        Spacer(Modifier.height(10.dp))
        Card {
            Column(modifier = Modifier.padding(12.dp)) {
                items.forEach { item ->
                    rows(item)
                }
            }
        }
    }
}

Funkcja `rows` jest przekazana jako parametr i definiuje sposób wyświetlania pojedynczego elementu `item` w kolumnie.

Zdefiniujmy teraz funkcję `Composable`, która będzie zawierać ui pojedynczego elementu listy. 

In [ ]:
@Composable
fun BaseRow(
    modifier: Modifier = Modifier,
    color: Color,
    title: String,
    subtitle: String,
    amount: Float,
    negative: Boolean
) {

Na elemencie chcemy wyświetlić tytuł, podtytuł oraz wartość. Oprócz tych informacji, przekazujemy informację (`negative`) o tym czy wyświetlane wartości mają być ujemne. Wpierw określmy znak dla wyświetlanej wartości.

In [ ]:
val sign = if (negative) "– " else " "

Następnie dodajmy i sformatujmy samą wartość.

In [ ]:
val formattedAmount = ("$sign${formatter.format(amount)} zł")

Elementy umieścimy w komponencie `Row`

In [ ]:
Row(
    modifier = modifier.height(68.dp),
    verticalAlignment = Alignment.CenterVertically
) {

Tworzymy pusty obszar o szerokości `4 dp` i wysokości `36 dp`, który jest wypełniony kolorem `color` powiązanym z danym rachunkiem.

In [ ]:
Spacer(modifier.size(4.dp, 36.dp).background(color = color))

Następnie dodajmy odstęp między paskiem koloru a polami tekstowymi.

In [ ]:
Spacer(Modifier.width(12.dp))

Dodajmy dwa pola tekstowe w kolumnie.

In [ ]:
Column(Modifier) {
    Text(text = title)
    Text(text = subtitle)
}

Po prawej stronie dodamy jeszcze jedno pole, wyświetlające kwotę. Wpierw tworzymy pusty obszar, który zajmuje dostępną przestrzeń po lewej stronie. W tym przypadku `Spacer` otrzymuje wagę (`weight`) równą `1f`, co oznacza, że zajmuje tyle przestrzeni, ile jest dostępne.

In [ ]:
Spacer(Modifier.weight(1f))

Na koniec dodajemy pole tekstowe wyświewtlające kwotę.

In [ ]:
Text(
    text = formattedAmount,
    modifier = Modifier.align(Alignment.CenterVertically),
    fontSize = 24.sp,
    fontWeight = FontWeight.Bold
)

Pełny kod:

In [ ]:
@Composable
fun BaseRow(
    modifier: Modifier = Modifier,
    color: Color,
    title: String,
    subtitle: String,
    amount: Float,
    negative: Boolean
) {
    val sign = if (negative) "– " else " "
    val formattedAmount = ("$sign${formatter.format(amount)} zł")
    Row(
        modifier = modifier.height(68.dp),
        verticalAlignment = Alignment.CenterVertically
    ) {
        Spacer(modifier.size(4.dp, 36.dp).background(color = color))
        Spacer(Modifier.width(12.dp))
        Column(Modifier) {
            Text(text = title)
            Text(text = subtitle)
        }
        Spacer(Modifier.weight(1f))
        Text(
            text = formattedAmount,
            modifier = Modifier.align(Alignment.CenterVertically),
            fontSize = 24.sp,
            fontWeight = FontWeight.Bold
        )
    }
}

Ostatnim elementem jest utworzenie `Composable` reprezentującej sam ekran rachunków.

In [ ]:
@Composable
fun BillsScreen() {
    val bills = remember { DataProvider.bills }
    StatementBody(
        modifier = Modifier.clearAndSetSemantics { contentDescription = "Bills" },
        items = bills,
        amounts = { bill -> bill.amount.toFloat() },
        colors = { bill -> bill.color },
        amountsTotal = bills.map { bill -> bill.amount.toFloat() }.sum(),
        circleLabel = "Total",
        rows = { bill ->
            BaseRow(
                title = bill.name,
                subtitle = bill.endDate.format(dateFormatter),
                amount = bill.amount.toFloat(),
                color = bill.color,
                negative = true
            )
        }
    )
}

Wykorzystujemy wcześniej zaimplementowane komponenty. `rows` - kompozycja wiersza, która określa jak wyświetlać każdy rachunek na liście. Jest to funkcja, która przyjmuje jeden rachunek i zwraca komponent `BaseRow`, który wyświetla informacje o rachunku.

Możemy przetestować apliakcję

<img src="https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExMGUxOTU0ZDE2NzgzZjljZjgwN2QzMzdlYTI1OThmZTYzNjgyY2M0MSZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/SsomlDGvA15rM2M2nz/giphy.gif" width="200" />

## Ekran kont

Dodajmy funkcję `Composable` dla ekranu kont - również tutaj wykorzystamy wcześniej utworzone funkcje.

In [ ]:
@Composable
fun AccountsScreen() {
    val amountsTotal = remember { DataProvider.accounts.sumOf { account -> account.amount }.toFloat() }
    StatementBody(
        modifier = Modifier.semantics { contentDescription = "Accounts Screen" },
        items = DataProvider.accounts,
        amounts = { account -> account.amount.toFloat() },
        colors = { account -> account.color },
        amountsTotal = amountsTotal,
        circleLabel = "Total",
        rows = { account ->
            BaseRow(
                title = account.name,
                subtitle = account.number.replaceRange(0 until 6, "******"),
                amount = account.amount.toFloat(),
                color = account.color,
                negative = false
            )
        }
    )
}

`subtitle` zawiera numer konta, który jest zamieniany na postać, w której pierwsze 6 cyfr zostaje zastąpione sześcioma gwiazdkami (`******`). Możemy przetestować aplikację.

<img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExMDE2YzdjYWNmZmYyYTNiYjA2OTAwOGZhMzI4Zjg3NjAwMDY5N2JjMSZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/F1dNJHm6OAPABjAVbk/giphy.gif" width="200" />

## Ekran przeglądowy

Nasz ekran będzie składał się z trzech sekcji
- pole wyświetlające powiadomienia
- pole wyświetlające stan i listę wszystkich kont
- pole wyświetlające stna i listę wszystkich rachunków

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExNmVlM2Q0M2I5YzM1OGZmOWQ4OWI1MzZjOWMxYjNmYTk1M2ZlY2Q1MyZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/KfMbjbJosqcsJhKcti/giphy.gif" width="200" />

### Pole Alerts

Rozpoczniemy od pola wyświetlającego komunikaty - nie będziemy dodawać żadnych komunikatów, po naciśnięciu przycisku aplikacja pokaże `DialogAlert` z informacją.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExNDQxYTQ3NDg1Zjg4YmYyNGVjY2EwYWM3YzBiNWUzOGY2MjJiNWEzYyZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/4caYanvGuNh0j3Hd2i/giphy.gif" width="200" />

Utwórzmy layout karty, na której rozmieścimy dwa pola `Text` oraz `Button`.

<img src="https://i.ibb.co/SQ3VCPB/Zrzut-ekra06-05-180748.png" width="300" />

Zewnętrznym elementem będzie `Row`

In [ ]:
@Composable
private fun Alert(onClickSeeAll: () -> Unit) {
    Row(
        modifier = Modifier
            .fillMaxWidth()
            .height(80.dp),
        horizontalArrangement = Arrangement.SpaceBetween
    ) {}

Następnie dwa pole `Text` umieszczamy w `Column`

In [ ]:
Column (
    modifier = Modifier.fillMaxHeight()
){
    Text(
        text = "Alerts",
        fontSize = 14.sp,
        modifier = Modifier.padding(start = 8.dp)
    )
    Text(
        text = "No Alerts !!!",
        fontSize = 20.sp,
        modifier = Modifier.padding(start = 12.dp, top = 16.dp),
        fontWeight = FontWeight.Bold
    )
}

Ostatnim elementem jest `Button`

In [ ]:
Button(
    onClick = onClickSeeAll,
    contentPadding = PaddingValues(0.dp),
    modifier = Modifier
        .align(Alignment.CenterVertically)
        .padding(end = 8.dp),
    shape = RectangleShape,
    colors = ButtonDefaults.buttonColors(containerColor = Color.Red)
) {
    Text(
        text = "SEE ALL"
    )
}

Pełna funkcja:

In [ ]:
@Composable
private fun Alert(onClickSeeAll: () -> Unit) {
    Row(
        modifier = Modifier
            .fillMaxWidth()
            .height(80.dp),
        horizontalArrangement = Arrangement.SpaceBetween
    ) {
        Column (
            modifier = Modifier.fillMaxHeight()
        ){
            Text(
                text = "Alerts",
                fontSize = 14.sp,
                modifier = Modifier.padding(start = 8.dp)
            )
            Text(
                text = "No Alerts !!!",
                fontSize = 20.sp,
                modifier = Modifier.padding(start = 12.dp, top = 16.dp),
                fontWeight = FontWeight.Bold
            )
        }

        Button(
            onClick = onClickSeeAll,
            contentPadding = PaddingValues(0.dp),
            modifier = Modifier
                .align(Alignment.CenterVertically)
                .padding(end = 8.dp),
            shape = RectangleShape,
            colors = ButtonDefaults.buttonColors(containerColor = Color.Red)
        ) {
            Text(
                text = "SEE ALL"
            )
        }
    }
}

Dodajmy teraz funkcję reprezentującą dialog, który zostanie wyświetlony po naciśnięciu przycisku

<img src="https://i.ibb.co/7ytpWxS/Zrzut-ekranu-2023-06-05-181651.png" width="300" />

In [ ]:
@Composable
private fun CustomAlertDialog(
    onDismiss: () -> Unit,
    buttonText: String
) {
    AlertDialog(
        onDismissRequest = onDismiss,
        text = { Text("No Alerts !!!") },
        confirmButton = {
            Column {
                Divider(
                    Modifier.padding(horizontal = 12.dp),
                    color = MaterialTheme.colorScheme.onSurface.copy(alpha = 0.2f)
                )
                TextButton(
                    onClick = onDismiss,
                    shape = RectangleShape,
                    contentPadding = PaddingValues(16.dp),
                    modifier = Modifier.fillMaxWidth()
                ) {
                    Text(buttonText)
                }
            }
        }
    )
}

Funkcja jako parametr przyjmuje funkcję `onDismiss` - dzięki niej zmieniamy wartość `showDialog` w funkcji `AlertCard`

In [ ]:
@Composable
private fun AlertCard() {
    var showDialog by remember { mutableStateOf(false) }

    if (showDialog) {
        CustomAlertDialog(
            onDismiss = { showDialog = false },
            buttonText = "Dismiss".uppercase(Locale.getDefault())
        )
    }
    Card {
        Column {
            Alert {
                showDialog = true
            }
        }
    }
}

Zmienna `showDialog` jest zmienną stanu (`mutableStateOf`) i jest używana do śledzenia stanu wyświetlania okna dialogowego. Początkowo ma wartość `false`, co oznacza, że okno dialogowe nie jest widoczne.

Jeśli `showDialog` ma wartość `true`, wtedy wyświetlane jest okno dialogowe `CustomAlertDialog`. Okno dialogowe jest zamknięte przez przekazanie funkcji do parametru `onDismiss`, która ustawia wartość `showDialog` na `false`.

### Pole Accounts oraz Bills

Ponieważ pole prezentujące dane kont oraz rachunków będzie wyglądać tak samo, rozpoczniemy od utworzenia komponentu generycznego.

<img src="https://i.ibb.co/grX33VW/Zrzut-ekranu-2023-06-05-182536.png" width="300" />

In [ ]:
@Composable
private fun <T> OverviewScreenCard(
    title: String,
    amount: Float,
    onClickSeeAll: () -> Unit,
    values: (T) -> Float,
    colors: (T) -> Color,
    data: List<T>,
    row: @Composable (T) -> Unit
) {

Parametry funkcji OverviewScreenCard to:
- `title` Tytuł karty
- `amount` Kwota, która będzie wyświetlana na karcie
- `onClickSeeAll` Funkcja obsługująca kliknięcie przycisku `See All`
- `values` Funkcja, która przyjmuje obiekt typu T i zwraca wartość liczbową (Float) dla tego obiektu, która będzie wykorzystywana w wierszach
- `colors` Funkcja, która przyjmuje obiekt typu T i zwraca kolor dla tego obiektu, który będzie wykorzystywany w wierszach
- `data` Lista obiektów typu `T`, które będą wykorzystywane do wygenerowania wierszy
- `row` Funkcja, która przyjmuje obiekt typu `T` i definiuje wygląd i zawartość wiersza na podstawie tego obiektu

Całość umieścimy w `Column`, pierwszą część zawierającą nazwę karty oraz kwotę całkowitą również umieścimy w `Column`

In [ ]:
Card {
    Column {
        Column {
            Text(
                text = title,
                style = MaterialTheme.typography.titleMedium,
                modifier = Modifier.padding(start = 8.dp)
            )
            val sign = if (data[0] is Bill) "- " else " "
            val amountText = "$sign${formatter.format(amount)} zł"
            Text(
                text = amountText,
                modifier = Modifier
                    .padding(top = 6.dp, bottom = 8.dp, start = 8.dp)
                    .fillMaxWidth(),
                fontSize = 36.sp,
                fontWeight = FontWeight.Bold,
                textAlign = TextAlign.Center
            )
        }
        ...
    }
}

Znak kwoty całkowitej tutaj ustalamy na podstawie typu danych. Następnym elementem będzie dodanie graficznej reprezentacji stanu kont - podobnie jak w `DonutChart`

In [ ]:
Card {
    Column {
        Column {
            ...
            OverViewDivider(data, values, colors)
            ...
    }
}

In [ ]:
@Composable
private fun <T> OverViewDivider(
    data: List<T>,
    values: (T) -> Float,
    colors: (T) -> Color
) {
    Row(Modifier.fillMaxWidth()) {
        data.forEach { item: T ->
            Spacer(
                modifier = Modifier
                    .weight(values(item))
                    .height(2.dp)
                    .background(colors(item))
            )
        }
    }
}

Komponent reprezentuje poziomy podział na ekranie przeglądu. Dzieli on przestrzeń na poziomej osi na podstawie dostarczonych danych i wartości liczbowych związanych z tymi danymi.
Parametry funkcji OverViewDivider to:
- `data` Lista obiektów typu `T`, które będą używane do wygenerowania podziału
- `values` Funkcja, która przyjmuje obiekt typu T i zwraca wartość liczbową dla tego obiektu, która będzie wykorzystywana do określenia szerokości podziału
- `colors` Funkcja, która przyjmuje obiekt typu T i zwraca kolor dla tego obiektu, który będzie wykorzystywany do określenia koloru podziału

Następnie dla każdego elementu w liście `data` generowany jest `Spacer` o wysokości `2.dp` i szerokości proporcjonalnej do wartości zwracanej przez funkcję `values(item)`. Kolor podziału jest określany na podstawie wartości zwracanej przez funkcję `colors(item)`.

Dodajmy ostatnie dwa elementy karty - listę kszystkich kont oraz przycisk umożliwiający przejście na odpowiedni ekran.

In [ ]:
Card {
    Column {
        Column {
            ...
            LazyColumn(modifier = Modifier.height(200.dp)){
                items(data.size){index ->
                    row(data[index])
                }
            }
            SeeAllButton(
                    modifier = Modifier.clearAndSetSemantics {
                        contentDescription = "All $title"
                    },
                    onClick = onClickSeeAll
            )
    }
}

W `LazyColumn` ustawiamy za pomocą modyfikatora wysokość listy - robimy to, ponieważ mamy zagnieżdżone przewijalne listy. Jeżeli nie ustalimy skończonej wysokości wewnętrznej listy, będzie ona interpretowana jako nieskończona, co jest niedozwolone - dzieje się tak ponieważ w przypadku dwóch zagnieżdżonych nieskończonych list `Compose` nie będzie w stanie określić, którą listę przewijać w przypadku wykrycia zdarzenia.

Na koniec dodajemy przycisk.

In [ ]:
@Composable
private fun SeeAllButton(modifier: Modifier = Modifier, onClick: () -> Unit) {
    TextButton(
        onClick = onClick,
        modifier = modifier
            .height(44.dp)
            .fillMaxWidth()
    ) {
        Text("SEE ALL")
    }
}

Pełny kod:

In [ ]:
@Composable
private fun <T> OverviewScreenCard(
    title: String,
    amount: Float,
    onClickSeeAll: () -> Unit,
    values: (T) -> Float,
    colors: (T) -> Color,
    data: List<T>,
    row: @Composable (T) -> Unit
) {
    Card {
        Column {
            Column {
                Text(
                    text = title,
                    style = MaterialTheme.typography.titleMedium,
                    modifier = Modifier.padding(start = 8.dp)
                )
                val sign = if (data[0] is Bill) "- " else " "
                val amountText = "$sign${formatter.format(amount)} zł"
                Text(
                    text = amountText,
                    modifier = Modifier
                        .padding(top = 6.dp, bottom = 8.dp, start = 8.dp)
                        .fillMaxWidth(),
                    fontSize = 36.sp,
                    fontWeight = FontWeight.Bold,
                    textAlign = TextAlign.Center
                )
            }
            OverViewDivider(data, values, colors)
            LazyColumn(modifier = Modifier.height(200.dp)){
                items(data.size){index ->
                    row(data[index])
                }
            }
            SeeAllButton(
                    modifier = Modifier.clearAndSetSemantics {
                        contentDescription = "All $title"
                    },
                    onClick = onClickSeeAll
                )
        }
    }
}

@Composable
private fun <T> OverViewDivider(
    data: List<T>,
    values: (T) -> Float,
    colors: (T) -> Color
) {
    Row(Modifier.fillMaxWidth()) {
        data.forEach { item: T ->
            Spacer(
                modifier = Modifier
                    .weight(values(item))
                    .height(2.dp)
                    .background(colors(item))
            )
        }
    }
}

@Composable
private fun SeeAllButton(modifier: Modifier = Modifier, onClick: () -> Unit) {
    TextButton(
        onClick = onClick,
        modifier = modifier
            .height(44.dp)
            .fillMaxWidth()
    ) {
        Text("SEE ALL")
    }
}

Ostatnim krokiem jest dodanie komponentów reprezentującego kartę kont, oraz rachunków

In [ ]:
@Composable
private fun AccountsCard(onClickSeeAll: () -> Unit) {
    val amount = DataProvider.accounts.sumOf { account -> account.amount }
    OverviewScreenCard(
        title = "Accounts",
        amount = amount.toFloat(),
        onClickSeeAll = onClickSeeAll,
        data = DataProvider.accounts,
        colors = { it.color },
        values = { it.amount.toFloat() }
    ) { account ->
        BaseRow(
            title = account.name,
            amount = account.amount.toFloat(),
            color = account.color,
            negative = false,
            subtitle = account.number.replaceRange(0 until 6, "******")
        )
    }
}

In [ ]:
@Composable
private fun BillsCard(onClickSeeAll: () -> Unit) {
    val amount = DataProvider.bills.sumOf { bill -> bill.amount }
    OverviewScreenCard(
        title = "Bills",
        amount = amount.toFloat(),
        onClickSeeAll = onClickSeeAll,
        data = DataProvider.bills,
        colors = { it.color },
        values = { it.amount.toFloat() }
    ) { bill ->
        BaseRow(
            color = bill.color,
            title = bill.name,
            subtitle = bill.endDate.toString().format(dateFormatter),
            amount = bill.amount.toFloat(),
            negative = true
        )
    }
}

W obu przypadkach widok każdego elementu wyświetlanej listy, tworzymy na podstawie funkcji przekazanej jako parametr od funkcji `OverviewScreenCard`, wywołujemy tutaj funkcję `BaseRow` (która została zaimplementowana we wcześniejszej częsci niniejszego opisu)

## Ekran przeglądowy

Pozostaje nam dodać komponent reprezentujący cały ekran

In [ ]:
@Composable
fun OverviewScreen(
    onClickSeeAllAccounts: () -> Unit = {},
    onClickSeeAllBills: () -> Unit = {}
) {
    Column(
        modifier = Modifier
            .padding(16.dp)
            .verticalScroll(rememberScrollState())
            .semantics { contentDescription = "Overview Screen" }
    ) {
        AlertCard()
        Spacer(Modifier.height(12.dp))
        AccountsCard(onClickSeeAll = onClickSeeAllAccounts)
        Spacer(Modifier.height(12.dp))
        BillsCard(onClickSeeAll = onClickSeeAllBills)
    }
}

Ponieważ przekazujemy dwie funkcje, musimy zmodyfikować `NavGraph`

In [ ]:
@Composable
fun NavGraph(navController: NavHostController, modifier: Modifier){
    NavHost(
        navController = navController,
        startDestination = Screens.Overview.route,
        modifier = modifier
    ) {
        composable(route = Screens.Overview.route){
            OverviewScreen(
                onClickSeeAllAccounts = { navController.navigateSingleTopTo(Screens.Accounts.route) },
                onClickSeeAllBills = {navController.navigateSingleTopTo(Screens.Bills.route)}
            ) }
        composable(route = Screens.Accounts.route){ AccountsScreen() }
        composable(route = Screens.Bills.route){ BillsScreen() }
    }
}

Wywołując funkcję `OverviewScreen` przekazujemy implementację fujnkcji `onClick`, dzięki którym przechodzimy na odpowiedni ekran.

<img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExMmRiMWVlZmIyYmRiMjEyYTdjMDU1MWI1OGM5MTI5ZjU4NzNjMTUyOSZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/TVVqv1DfADRPbKWN4a/giphy.gif" width="300" />